# Submission Guide



*   After completion of implementation, you should run all the cells
*   Submit your ipython notebook in 'ipynb' format
    * **Do not remove your output results from every cells**
*   File name format: lab1_studentID_name.
ipynb
    * Ex) lab1_20233809_MinsungHwang.ipynb
*   Submission due: March 26th by 10:00AM

    * **We do not accept late submissions**



# Load Dataset
You can upload "nodes.txt" and "edges.txt" at once

In [1]:
from google.colab import files
f = files.upload()

Saving edges.txt to edges.txt
Saving nodes.txt to nodes.txt


Preprocess the uploaded files

In [16]:
node_list = f['nodes.txt'].decode('utf-8').strip().split('\n')
node_to_id = dict(zip(node_list, range(len(node_list))))

edge_lines = f['edges.txt'].decode('utf-8').strip().split('\n')
edge_list = [(edge.split()[0], edge.split()[1]) for edge in edge_lines]

print(f"# Nodes: {len(node_list)}")
print(f"# Edges: {len(edge_list)}")

# Nodes: 7363
# Edges: 35146


# Computing Personalized PageRanks (Approach 1)

In [23]:
from typing import List

class Node:

    def __init__(self, id: str) -> None:

        self.id: str = id
        self.in_neighbors: List[Node] = []
        self.out_neighbors: List[Node] = []

        self.pagerank: float = 0
        self.pagerank_next: float = 0
        self.personalized: float = 0

    @property
    def out_degree(self) -> int:
        return len(self.out_neighbors)

    # TODO: Compute Personalized PageRank of a single node
    def aggregate_pagerank(self, alpha: float) -> None:
        self.pagerank_next = alpha * sum((node.pagerank / node.out_degree) for node in self.in_neighbors) + self.personalized

    def update_pagerank(self) -> None:
        self.pagerank = self.pagerank_next

In [24]:
from typing import Tuple, Dict

class Graph:

    def __init__(self, node_list: List[str], edge_list: List[Tuple[str, str]]):

        self.nodes: Dict[str, Node] = {}
        self.edges = edge_list

        self._build_graph(node_list)
        self.num_nodes = len(self.nodes)

    def _build_graph(self, node_list: List[str]):

        for id in node_list:
            node = Node(id)
            self.nodes[id] = node

        for edge in self.edges:
            id_head = edge[0]
            id_tail = edge[1]

            self.nodes[id_head].out_neighbors.append(self.nodes[id_tail])
            self.nodes[id_tail].in_neighbors.append(self.nodes[id_head])

In [25]:
from typing import Optional

def compute_pagerank(node_list: List[str], edge_list: List[Tuple[str, str]],
                     max_iters: int, alpha: float, tolerance: float,
                     personalize_list: Optional[List[str]] = None) -> Dict[str, float]:

    graph = Graph(node_list, edge_list)

    # TODO: Compute the Personalized PageRank with the predefined list
    if personalize_list is None:
      personalize_list = graph.nodes.keys()
    norm = len(personalize_list)

    nodes = graph.nodes.values()
    for node in nodes:
        if node.id in personalize_list:
          node.pagerank = (1-alpha) / norm
          node.personalized = (1-alpha) / norm

    for iter in range(max_iters):

        # TODO: Implement one iteration
        prev_pageranks = [node.pagerank for node in graph.nodes.values()]

        for node in nodes:
          node.aggregate_pagerank(alpha)

        for node in nodes:
          node.update_pagerank()

        curr_pageranks = [node.pagerank for node in graph.nodes.values()]

        error = max(abs(prev-curr) for prev, curr in zip(prev_pageranks, curr_pageranks))
        if error < tolerance:
          print("Total iterations : ", iter)
          break
        else:
          print("It reaches the maximum iterations. Please increase the maxiters.")
    pageranks = [node.pagerank for node in graph.nodes.values()]
    pageranks = dict(zip(graph.nodes, pageranks))

    return pageranks

In [26]:
from prettytable import PrettyTable

def print_pagerank_top10(pageranks: Dict[str, float], name='PageRank') -> None:

    pageranks_sorted = sorted(pageranks.items(), reverse=True, key=lambda x: x[1])[:10]

    table = PrettyTable(field_names = ['Node ID', name])
    for id, score in pageranks_sorted[:10]:
        table.add_row([id, round(score, 4)])
    print(table)

In [27]:
pageranks = compute_pagerank(node_list, edge_list, 10000, 0.85, 1e-8)
print_pagerank_top10(pageranks)

It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum it

In [28]:
# Set the predefined set in the form of a list
personalize_list = ['politicianus:joe_biden', 'politicianus:biden', 'politicianus:senator_biden']
personalized_pageranks = compute_pagerank(node_list, edge_list, 10000, 0.85, 1e-8,
                                          personalize_list=personalize_list)
print_pagerank_top10(personalized_pageranks, name='PPR')

It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum it

# Computing Personalized PageRanks with Sparse Matrix (Approach 2)

In [37]:
import numpy as np
from scipy.sparse import diags, coo_matrix

def compute_pagerank_with_sparse_matrix(node_list: List[str], edge_list: List[Tuple[str, str]], node_to_id: Dict[str, int],
                                        max_iters: int, alpha: float, tolerance: float,
                                        personalize_list: Optional[List[str]] = None) -> Dict[str, float]:

    # TODO: Compute the Personalized PageRank with the predefined list using sparse matrix-vector multiplication
    edge_list = [(node_to_id[edge[0]], node_to_id[edge[1]]) for edge in edge_list]

    num_nodes = len(node_list)
    rows, cols, data = zip(*[(head, tail, 1) for head, tail in edge_list])
    adj = coo_matrix((data, (rows, cols)), shape = (num_nodes, num_nodes), dtype = float)

    degrees = adj @ np.ones(num_nodes)
    inv_degrees = np.reciprocal(degrees)
    D_inv = diags(inv_degrees, shape = (num_nodes, num_nodes))
    P = D_inv @ adj
    PT = P.transpose()

    if personalize_list is None:
      personalize_vector = np.ones((num_nodes, 1), dtype = float)
    else:
      node_ids = np.array(list(map(node_to_id.get, personalize_list)), dtype = int)
      personalize_vector = np.zeros((num_nodes, 1), dtype = float)
      personalize_vector[node_ids] = 1

    personalize_vector /= np.sum(personalize_vector)
    personalize_vector = (1 - alpha) * personalize_vector
    pageranks = np.copy(personalize_vector)


    for iter in range(max_iters):

        # TODO: Implement power iteration
        prev_pageranks = pageranks
        pageranks = alpha * PT @ pageranks + personalize_vector

        error = max(np.absolute(prev_pageranks - pageranks))
        if error < tolerance:
          print("Total iterations : ", iter)
          break
        else:
          print("It reaches the maximum iterations. Please increase the maxiters.")

    pageranks = np.asarray(pageranks)
    pageranks = dict(zip(node_list, pageranks.squeeze().tolist()))

    return pageranks

In [38]:
pageranks = compute_pagerank_with_sparse_matrix(node_list, edge_list, node_to_id, 10000, 0.85, 1e-8)
print_pagerank_top10(pageranks)

It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum it

In [39]:
personalized_pageranks = compute_pagerank_with_sparse_matrix(node_list, edge_list, node_to_id, 10000,
                                                             0.85, 1e-8, personalize_list=personalize_list)
print_pagerank_top10(personalized_pageranks, name = 'PPR')

It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum iterations. Please increase the maxiters.
It reaches the maximum it